In [1]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel, AutoImageProcessor
from PIL import Image
import requests

processor = AutoImageProcessor.from_pretrained("nomic-ai/nomic-embed-vision-v1.5")
vision_model = AutoModel.from_pretrained("nomic-ai/nomic-embed-vision-v1.5", trust_remote_code=True)

preprocessor_config.json:   0%|          | 0.00/791 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


config.json: 0.00B [00:00, ?B/s]

configuration_hf_nomic_bert.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/nomic-ai/nomic-bert-2048:
- configuration_hf_nomic_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_hf_nomic_bert.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/nomic-ai/nomic-bert-2048:
- modeling_hf_nomic_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/372M [00:00<?, ?B/s]

In [13]:
import os
from tqdm import tqdm
prompt = "flower"
path = f"erbario_plant_images_{prompt}_cleaned"
files = os.listdir(path)
stuff_to_save = []
for f in tqdm(files):
  image = Image.open(f"{path}/{f}")
  inputs = processor(image, return_tensors="pt")
  with torch.no_grad():
    img_emb = vision_model(**inputs).last_hidden_state.cpu()
    img_embeddings = F.normalize(img_emb[:, 0], p=2, dim=1)
    stuff_to_save.append({"file": f, "emb": img_embeddings})
    if len(stuff_to_save) % 10 == 0:
      torch.cuda.empty_cache()

100%|██████████| 209/209 [00:26<00:00,  7.78it/s]


In [14]:
import pickle as pkl
with open(f"erbario_plant_images_{prompt}.pkl", "wb") as f:
  pkl.dump(stuff_to_save, f)